In [67]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
%matplotlib inline
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

In [31]:
train = pd.read_csv('data/train_set.csv')
test = pd.read_csv('data/test_set.csv')

train.rename(columns={ 'pos_adress_lat' : 'pos_address_lat', 
                       'pos_adress_lon' : 'pos_address_lon'}, inplace=True)

D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
# Разделение трейна на X и Y
X = train.drop(['home_add_lat', 'home_add_lon','work_add_lat', 'work_add_lon'], axis=1)
Y = train[['home_add_lat', 'home_add_lon','work_add_lat', 'work_add_lon']]

In [33]:
X.shape

(1224734, 14)

In [34]:
Y.shape

(1224734, 4)

In [35]:
# Объединение трейна и теста для общего labelencodera и onehotencodera
train_test = pd.concat([X, test])

In [36]:
X.shape, test.shape, X.shape[0]+test.shape[0], train_test.shape

((1224734, 14), (1265470, 14), 2490204, (2490204, 14))

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1224734 non-null float64
atm_address         269473 non-null object
atm_address_lat     184680 non-null float64
atm_address_lon     184680 non-null float64
city                1224725 non-null object
country             1224734 non-null object
currency            1224690 non-null float64
customer_id         1224734 non-null object
home_add_lat        1224718 non-null float64
home_add_lon        1224718 non-null float64
mcc                 1224734 non-null int64
pos_address         924991 non-null object
pos_address_lat     942614 non-null float64
pos_address_lon     942614 non-null float64
terminal_id         1224705 non-null object
transaction_date    1224690 non-null object
work_add_lat        664660 non-null float64
work_add_lon        664660 non-null float64
dtypes: float64(10), int64(1), object(7)
memory usage: 168.2+ MB


In [38]:
#train.info()
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2490204 entries, 0 to 1265469
Data columns (total 14 columns):
amount              float64
atm_address         object
atm_address_lat     float64
atm_address_lon     float64
city                object
country             object
currency            float64
customer_id         object
mcc                 object
pos_address         object
pos_address_lat     float64
pos_address_lon     float64
terminal_id         object
transaction_date    object
dtypes: float64(6), object(8)
memory usage: 285.0+ MB


In [43]:
# Категорийные признаки
categorical_columns = train.columns[train.dtypes == 'object']

In [44]:
#train[categorical_columns] = train[categorical_columns].fillna(value = '0')
train_test[categorical_columns] = train_test[categorical_columns].fillna(value = '0')

In [218]:
len(train_test['customer_id'].unique())

19997

In [45]:
#train.info()

In [47]:
label_encoder = LabelEncoder()

In [48]:
for column in categorical_columns:
    train_test[column] = label_encoder.fit_transform(train_test[column])

In [50]:
train_test[:5]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date
0,2.884034,1547,NaN,NaN,7805,129,643.0,1217,5261,2960,59.844072,30.179153,19895,170
1,2.775633,1547,NaN,NaN,7805,129,643.0,1217,5261,2960,59.844072,30.179153,264575,274
2,3.708368,1547,NaN,NaN,8179,129,643.0,1217,5992,104740,59.858198,30.229024,252847,250
3,2.787498,1547,NaN,NaN,7805,129,643.0,1217,5261,2960,59.844072,30.179153,122687,226
4,2.892510,1547,NaN,NaN,7805,129,643.0,1217,5261,2960,59.844072,30.179153,6310,161


In [60]:
# Обратное разделение
X1 = train_test[:1224734]
X_test = train_test[1224734:]

In [56]:
X1_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224734 entries, 0 to 1224733
Data columns (total 14 columns):
amount              1224734 non-null float64
atm_address         1224734 non-null int64
atm_address_lat     184680 non-null float64
atm_address_lon     184680 non-null float64
city                1224734 non-null int64
country             1224734 non-null int64
currency            1224690 non-null float64
customer_id         1224734 non-null int64
mcc                 1224734 non-null object
pos_address         1224734 non-null int64
pos_address_lat     942614 non-null float64
pos_address_lon     942614 non-null float64
terminal_id         1224734 non-null int64
transaction_date    1224734 non-null int64
dtypes: float64(6), int64(7), object(1)
memory usage: 140.2+ MB


In [57]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 4 columns):
home_add_lat    1224718 non-null float64
home_add_lon    1224718 non-null float64
work_add_lat    664660 non-null float64
work_add_lon    664660 non-null float64
dtypes: float64(4)
memory usage: 37.4 MB


In [58]:
# Замена нулями в Y (возможно не самый лучший вариант)
Y = Y.fillna(value=0)

In [59]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 4 columns):
home_add_lat    1224734 non-null float64
home_add_lon    1224734 non-null float64
work_add_lat    1224734 non-null float64
work_add_lon    1224734 non-null float64
dtypes: float64(4)
memory usage: 37.4 MB


In [85]:
Y.head()

,home_add_lat,home_add_lon,work_add_lat,work_add_lon
0,59.851,30.232,59.847,30.177
1,59.851,30.232,59.847,30.177
2,59.851,30.232,59.847,30.177
3,59.851,30.232,59.847,30.177
4,59.851,30.232,59.847,30.177


In [76]:
X1 = X1.fillna(value=0)

In [87]:
X1.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date
0,2.884034,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,19895,170
1,2.775633,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,264575,274
2,3.708368,1547,0.0,0.0,8179,129,643.0,1217,5992,104740,59.858198,30.229024,252847,250
3,2.787498,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,122687,226
4,2.892510,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,6310,161


In [88]:
train_test_split?

In [90]:
# Разделение на трейн и валидацию
X_train, X_valid, Y_train, Y_valid = train_test_split(X1, Y, test_size=0.2, random_state=42, shuffle=False)

In [91]:
X_train.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date
0,2.884034,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,19895,170
1,2.775633,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,264575,274
2,3.708368,1547,0.0,0.0,8179,129,643.0,1217,5992,104740,59.858198,30.229024,252847,250
3,2.787498,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,122687,226
4,2.892510,1547,0.0,0.0,7805,129,643.0,1217,5261,2960,59.844072,30.179153,6310,161


In [92]:
Y_train.head()

,home_add_lat,home_add_lon,work_add_lat,work_add_lon
0,59.851,30.232,59.847,30.177
1,59.851,30.232,59.847,30.177
2,59.851,30.232,59.847,30.177
3,59.851,30.232,59.847,30.177
4,59.851,30.232,59.847,30.177


In [93]:
Y_home_lat_train, Y_home_lat_valid = Y_train['home_add_lat'], Y_valid['home_add_lat']
Y_home_lon_train, Y_home_lon_valid = Y_train['home_add_lon'], Y_valid['home_add_lon']
Y_work_lat_train, Y_work_lat_valid = Y_train['work_add_lat'], Y_valid['work_add_lat']
Y_work_lon_train, Y_work_lon_valid = Y_train['work_add_lon'], Y_valid['work_add_lon']

In [126]:
reg1 = LinearRegression(normalize=True,n_jobs=-1)
reg1.fit(X_train, Y_home_lat_train)
score1 = reg1.score(X_valid, Y_home_lat_valid)
print(score1)

-1.98134494053


In [123]:
reg2 = LinearRegression(normalize=True,n_jobs=-1)
reg2.fit(X_train, Y_home_lon_train)
score2 = reg2.score(X_valid, Y_home_lon_valid)
print(score2)

0.348945759295


In [124]:
reg3 = LinearRegression(normalize=True,n_jobs=-1)
reg3.fit(X_train, Y_work_lat_train)
score3 = reg3.score(X_valid, Y_work_lat_valid)
print(score3)

-0.0243447397005


In [125]:
reg4 = LinearRegression(normalize=True,n_jobs=-1)
reg4.fit(X_train, Y_work_lon_train)
score4 = reg4.score(X_valid, Y_work_lon_valid)
print(score4)

0.0206550859736


In [140]:
X_test = X_test.fillna(value=0)

In [141]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1265470 entries, 0 to 1265469
Data columns (total 14 columns):
amount              1265470 non-null float64
atm_address         1265470 non-null int64
atm_address_lat     1265470 non-null float64
atm_address_lon     1265470 non-null float64
city                1265470 non-null int64
country             1265470 non-null int64
currency            1265470 non-null float64
customer_id         1265470 non-null int64
mcc                 1265470 non-null float64
pos_address         1265470 non-null int64
pos_address_lat     1265470 non-null float64
pos_address_lon     1265470 non-null float64
terminal_id         1265470 non-null int64
transaction_date    1265470 non-null int64
dtypes: float64(7), int64(7)
memory usage: 184.8 MB


In [133]:
X_test['mcc'] = X_test['mcc'].str.replace(",","").astype(float)

In [134]:
X_test['mcc'].head()

0    5411.0
1    4111.0
2    5411.0
3    5411.0
4    5814.0
Name: mcc, dtype: float64

In [136]:
pred = reg1.predict(X_train)

In [137]:
reg1.score(X_train, Y_home_lat_train)

0.31226662753812839

In [ ]:
X_test = X_test.

In [143]:
pred1 = reg1.predict(X_test)
pred2 = reg2.predict(X_test)
pred3 = reg3.predict(X_test)
pred4 = reg4.predict(X_test)

In [144]:
pred1

array([ 55.57804735,  55.23114428,  56.75879042, ...,  41.09804458,
        50.6109604 ,  49.59304147])

In [152]:
pred1 = np.round(pred1, 3)    # home_lat
pred2 = np.round(pred2, 3)    # home_lon
pred3 = np.round(pred3, 3)    # work_lat
pred4 = np.round(pred4, 3)    # work_lon

In [217]:
pred1

array([ 55.578,  55.231,  56.759, ...,  41.098,  50.611,  49.593])

In [145]:
Y_train.head()

,home_add_lat,home_add_lon,work_add_lat,work_add_lon
0,59.851,30.232,59.847,30.177
1,59.851,30.232,59.847,30.177
2,59.851,30.232,59.847,30.177
3,59.851,30.232,59.847,30.177
4,59.851,30.232,59.847,30.177


In [177]:
submit = pd.concat([pd.Series(pred3, index=test['customer_id']), 
                    pd.Series(pred4, index=test['customer_id']),
                    pd.Series(pred1, index=test['customer_id']), 
                    pd.Series(pred2, index=test['customer_id'])],axis=1)

In [178]:
submit.head()

,0,1,2,3
customer_id,,,,
00fd410f5c580c8351cafa88d82b60f3,29.359,20.859,55.578,38.919
00fd410f5c580c8351cafa88d82b60f3,26.581,18.883,55.231,38.349
00fd410f5c580c8351cafa88d82b60f3,31.584,21.395,56.759,38.597
00fd410f5c580c8351cafa88d82b60f3,29.313,20.811,55.576,38.862
00fd410f5c580c8351cafa88d82b60f3,28.442,19.730,55.978,38.588


In [179]:
submit.columns = ['work_add_lat','work_add_lon','home_add_lat','home_add_lon']
submit.head()

,work_add_lat,work_add_lon,home_add_lat,home_add_lon
customer_id,,,,
00fd410f5c580c8351cafa88d82b60f3,29.359,20.859,55.578,38.919
00fd410f5c580c8351cafa88d82b60f3,26.581,18.883,55.231,38.349
00fd410f5c580c8351cafa88d82b60f3,31.584,21.395,56.759,38.597
00fd410f5c580c8351cafa88d82b60f3,29.313,20.811,55.576,38.862
00fd410f5c580c8351cafa88d82b60f3,28.442,19.730,55.978,38.588


In [175]:
submit.index?

In [180]:
submit.to_csv('submit1.csv')

In [182]:
len(test['customer_id'].unique())

9997

In [190]:
submit.index

Index(['00fd410f5c580c8351cafa88d82b60f3', '00fd410f5c580c8351cafa88d82b60f3',
       '00fd410f5c580c8351cafa88d82b60f3', '00fd410f5c580c8351cafa88d82b60f3',
       '00fd410f5c580c8351cafa88d82b60f3', '00fd410f5c580c8351cafa88d82b60f3',
       '00fd410f5c580c8351cafa88d82b60f3', '00fd410f5c580c8351cafa88d82b60f3',
       '00fd410f5c580c8351cafa88d82b60f3', '00fd410f5c580c8351cafa88d82b60f3',
       ...
       '3504d15e1000658c22db9e974956fd26', '3504d15e1000658c22db9e974956fd26',
       '159673c2adc0b2be66aa48e9f80f738a', '159673c2adc0b2be66aa48e9f80f738a',
       '159673c2adc0b2be66aa48e9f80f738a', '159673c2adc0b2be66aa48e9f80f738a',
       '9c3bb46e374df29c69b44a19cdbf929b', '90309ee8223cddc27349e9f32eb8ad1e',
       '3696f3e678c786b53f8aecd9e1a7b647', '213e6bbf6548701df2a23762c92da044'],
      dtype='object', name='customer_id', length=1265470)

In [184]:
submit.groupby('customer_id').count_values()

AttributeError: 'DataFrameGroupBy' object has no attribute 'count_values'

In [191]:
sample = pd.read_csv('sample.csv')

In [192]:
sample.head()

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00021683ccb416637fe9a4cd35e4606e,0,0,0,0
1,0002d0f8a642272b41c292c12ab6e602,0,0,0,0
2,0004d182d9fede3ba2534b2d5e5ad27e,0,0,0,0
3,0008c2445518c9392cb356c5c3db3392,0,0,0,0
4,000b373cc4969c0be8e0933c08da67e1,0,0,0,0


In [193]:
sample2 = pd.read_csv('submit1.csv')
sample2.head()

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,29.359,20.859,55.578,38.919
1,00fd410f5c580c8351cafa88d82b60f3,26.581,18.883,55.231,38.349
2,00fd410f5c580c8351cafa88d82b60f3,31.584,21.395,56.759,38.597
3,00fd410f5c580c8351cafa88d82b60f3,29.313,20.811,55.576,38.862
4,00fd410f5c580c8351cafa88d82b60f3,28.442,19.730,55.978,38.588


In [194]:
sample.to_csv('sample1.csv')

In [198]:
submit.index[0]

'00fd410f5c580c8351cafa88d82b60f3'

In [201]:
submit1 = pd.concat([test['customer_id'],
                    pd.Series(pred3), 
                    pd.Series(pred4),
                    pd.Series(pred1), 
                    pd.Series(pred2)],axis=1)

In [206]:
submit1.columns = ['customer_id','work_add_lat','work_add_lon','home_add_lat','home_add_lon']
submit1.head()

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,29.359,20.859,55.578,38.919
1,00fd410f5c580c8351cafa88d82b60f3,26.581,18.883,55.231,38.349
2,00fd410f5c580c8351cafa88d82b60f3,31.584,21.395,56.759,38.597
3,00fd410f5c580c8351cafa88d82b60f3,29.313,20.811,55.576,38.862
4,00fd410f5c580c8351cafa88d82b60f3,28.442,19.730,55.978,38.588


In [207]:
submit1.shape

(1265470, 5)

In [208]:
sample.shape

(9997, 5)

In [211]:
sample['customer_id'][0]

'00021683ccb416637fe9a4cd35e4606e'

In [215]:
submit1 = submit1.groupby('customer_id').mean()

In [216]:
submit1.to_csv('submit1.csv')

In [212]:
submit1.loc[submit1['customer_id'] == sample['customer_id'][0]];

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
802502,00021683ccb416637fe9a4cd35e4606e,35.665,44.514,54.327,73.504
802503,00021683ccb416637fe9a4cd35e4606e,31.487,41.810,54.067,73.495
802504,00021683ccb416637fe9a4cd35e4606e,36.182,44.955,54.207,73.559
802505,00021683ccb416637fe9a4cd35e4606e,32.941,43.794,54.050,75.497
802506,00021683ccb416637fe9a4cd35e4606e,33.316,43.097,53.975,73.498
802507,00021683ccb416637fe9a4cd35e4606e,32.765,42.612,54.002,73.324
802508,00021683ccb416637fe9a4cd35e4606e,35.006,44.164,54.410,73.772
802509,00021683ccb416637fe9a4cd35e4606e,33.471,40.850,54.261,69.398
802510,00021683ccb416637fe9a4cd35e4606e,35.457,44.274,53.968,72.963
802511,00021683ccb416637fe9a4cd35e4606e,34.012,43.486,54.335,73.668


In [210]:
np.where(sample['customer_id'] == submit1['customer_id'])

ValueError: Can only compare identically-labeled Series objects